In [ ]:
import numpy as np
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from getDailyData import get_daily
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression  # Example model
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from lightgbm import LGBMRegressor
from tqdm import tqdm


In [ ]:
import yfinance as yf
spx = yf.Ticker('^GSPC')
spx.history(start='2023-11-20', interval='1d')

In [ ]:
import datetime
from datetime import time, timedelta
from tqdm import tqdm

now = datetime.datetime.now()
df_consolidated = pd.DataFrame()
results = {}
coefs = {}

morning_start = datetime.datetime.combine(now.date(), time(6, 30))
delta = now - morning_start
print(delta)
# candle = 1 #max(0,min((delta.total_seconds() / 60 / 30) // 1, 12))
# candles = np.arange(1,13)
candles = np.arange(1,2)
for candle in tqdm(candles):
    print(f'running for {str(candle)}')
    data, df_final, final_row = get_daily(mode='intra', periods_30m=candle)

    df_new = data[['Open','High','Low','Close','Close30','Close_VIX30','Close_VIX','Close_VVIX30','Close_VVIX']].copy()
    df_new['PrevClose'] = df_new['Close'].shift(1)
    df_new['CurrentGap'] = (df_new['Open'] / df_new['PrevClose']) - 1
    df_new['ClosePctIntra'] = (df_new['Close30'] / df_new['Close'].shift(1)) - 1
    df_new['ClosePctOpenIntra'] = (df_new['Close30'] / df_new['Open']) - 1
    df_new['ClosePctVIXIntra'] = (df_new['Close_VIX30'] / df_new['Close_VIX'].shift(1)) - 1
    df_new['ClosePctVVIXIntra'] = (df_new['Close_VVIX30'] / df_new['Close_VVIX'].shift(1)) - 1
    df_new['EMA8'] = df_new['Close'].ewm(8).mean()
    df_new['EMA8'] = df_new['EMA8'].shift(1)
    df_new['EMA8Intra'] = df_new['Close30'] > df_new['EMA8']

    # Target will be the day's close
    df_new['ClosePct'] = (df_new['Close'] / df_new['Close'].shift(1)) - 1

    # Column to determine what percentile the current intra performance looks like
    intra_rank = []
    for i, pct in tqdm(enumerate(df_new['ClosePctIntra'])):
        try:
            historical = df_new['ClosePctIntra'].iloc[:i]
            current = df_new['ClosePctIntra'].iloc[i]
            perc = len(historical[historical > current]) / len(historical)
        except:
            perc = None
        intra_rank.append(perc)

    df_new['IntraPercentile'] = intra_rank

    # Column to determine what percentile the daily performance looks like
    daily_rank = []
    for i, pct in tqdm(enumerate(df_new['ClosePct'])):
        try:
            historical = df_new['ClosePct'].iloc[:i]
            current = df_new['ClosePct'].iloc[i]
            perc = len(historical[historical > current]) / len(historical)
        except:
            perc = None
        daily_rank.append(perc)

    df_new['ClosePctPercentile'] = daily_rank

    # Let's do n-5 to start just for closes
    lags = np.arange(1,6)

    for lag in lags:
        df_new[f'ClosePct_n{str(lag)}'] = df_new['ClosePct'].shift(lag)
        # df_new[f'ClosePctPercentile_n{str(lag)}'] = df_new['ClosePctPercentile'].shift(lag)


    df_feats = df_new[[c for c in df_new.columns if 'ClosePct' in c or 'Intra' in c or 'Gap' in c]]

    df_final = df_feats.dropna()

    X = df_final[['ClosePctIntra']]  # Feature dataset
    y = df_final['ClosePct']    # Target dataset

    # model = LGBMRegressor(random_state=42, n_estimators=10, verbose=-1)
    # model = LinearRegression()
    # Define the column transformer for handling numeric and categorical features
    

    # Fit the pipeline on the training data
    # pipeline.fit(X_train, y_train)

    tscv = TimeSeriesSplit(n_splits=len(df_final)-1, max_train_size=None, test_size=1)

    mae_scores = []
    overall_results = []

    for train_index, test_index in tscv.split(X):
        
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        
        # Select features
        categorical_features = X_train.select_dtypes(include='object').columns
        numeric_features = X_train.drop(columns=[c for c in X_train.columns if 'Percentile' in c]).select_dtypes(include='number').columns

        # Transformers
        numeric_transformer = RobustScaler()  # Example: StandardScaler for numeric features
        categorical_transformer = OneHotEncoder()  # Example: OneHotEncoder for categorical features

        # Define the pipeline steps
        preprocessor = ColumnTransformer(
            transformers=[
                ('numeric', numeric_transformer, numeric_features),  # numeric_features is a list of numeric feature column names
                ('categorical', categorical_transformer, categorical_features)  # categorical_features is a list of categorical feature column names
            ])

        # Create the pipeline
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', LinearRegression())
        ])
        
        # Fit the model
        pipeline.fit(X_train, y_train)

        # Predict
        y_pred = pipeline.predict(X_test)

        # Calculate metrics
        # mae_scores.append(mean_absolute_error(y_test, y_pred))
        result_df = pd.DataFrame({'IsTrue': y_test, 'Predicted': y_pred}, index=y_test.index)
        overall_results.append(result_df)

    df_results = pd.concat(overall_results)

    uppers = []
    lowers = []
    alpha = 0.05
    for i, pct in tqdm(enumerate(df_results['Predicted']), desc='Calibrating Probas',total=len(df_results)):
        try:
            
            df_q = df_results.iloc[:i]
            pred = df_results['Predicted'].iloc[-1]
            errors = df_q['IsTrue'] - df_q['Predicted']
            positive_errors = errors[errors >= 0]
            negative_errors = errors[errors < 0]

            # Calculate bounds
            upper_bound = pred + np.quantile(positive_errors, 1 - alpha)
            lower_bound = pred + np.quantile(negative_errors, alpha)
            
        except:
            upper_bound = None
            lower_bound = None

        uppers.append(upper_bound)
        lowers.append(lower_bound)

    df_results['Upper'] = uppers
    df_results['Lower'] = lowers

    df_results = df_results.merge(data[['PrevClose']],left_index=True, right_index=True)
    df_results['Pred'] = df_results['PrevClose'] * (1 + df_results['Predicted'])
    df_results['Actual'] = df_results['PrevClose'] * (1 + df_results['IsTrue'])
    df_results['Up'] = df_results['PrevClose'] * (1 + df_results['Upper'])
    df_results['Down'] = df_results['PrevClose'] * (1 + df_results['Lower'])

    results[f'{str(int(candle))}'] = df_results

    # Average metrics across folds
    average_mae = mean_absolute_error(df_results['IsTrue'], df_results['Predicted'])
    # sorted_features = sorted([(feat, coef) for feat, coef in zip(model.feature_name_, model.feature_importances_)], key=lambda x: abs(x[1]), reverse=True)
    sorted_features = sorted([(feat, coef) for feat, coef in zip(pipeline.feature_names_in_, pipeline.named_steps.model.coef_)], key=lambda x: abs(x[1]), reverse=True)

    coefs[f'{str(int(candle))}'] = pd.DataFrame(sorted_features, columns=['Feature','Coefficient'])

    df_consolidated.loc[int(candle), 'MAE'] = average_mae

In [ ]:
pipeline.named_steps['model'].coef_

In [ ]:
df_f = pd.concat(coefs)

In [ ]:
df_consolidated

In [ ]:
results[f'{str(candle)}'].loc['2023-10-01':, ['Pred','Actual','Up','Down']].plot();

In [ ]:
coefs[f'{str(candle)}']

In [3]:
from getDailyData import get_daily
from model_intra_v2 import walk_forward_validation
from model_day_v2 import walk_forward_validation_seq as walk_forward_validation_daily
from model_regr_v2 import walk_forward_validation as walk_forward_validation_regr
from model_regr_v2 import calc_upper_lower
import pandas as pd
import json
from dbConn import connection, engine, insert_dataframe_to_sql
import numpy as np
from datetime import time, timedelta
import datetime
from pandas.tseries.offsets import BDay
import holidays
from dotenv import load_dotenv
load_dotenv()

periods_30m = 1

if periods_30m > 0:
    data, df_final, final_row = get_daily(mode='intra', periods_30m=periods_30m)
    # Regression model
    res, _ = walk_forward_validation(df_final.drop(columns=['Target']).dropna(), 'Target_clf', 1, mode='single')
    regr_res, _ = walk_forward_validation_regr(df_final[['CurrentClose30toClose','ClosePct']].dropna(), 'ClosePct', 1, mode='single')
    df_regr_results = pd.read_sql_query(f'select * from reg_results where ModelNum = {str(periods_30m)}', con = engine)
    regr_pct = regr_res['Predicted'].iloc[-1]
    upper, lower = calc_upper_lower(regr_pct, df_regr_results, alpha=0.05)

elif periods_30m == 0:
    data, df_final, final_row = get_daily()
    res, _, _ = walk_forward_validation_daily(df_final.dropna(), 'Target_clf', 'Target', 200, 1)

# Get results, run calibration and pvalue    

df_results = pd.read_sql_query(f'select * from results where ModelNum = {str(periods_30m)}', con = engine)

# Calibrate Probabilities
def get_quantiles(df, col_name, q):
    return df.groupby(pd.cut(df[col_name], q))['IsTrue'].mean()

pct = res['Predicted'].iloc[-1]

df_q = get_quantiles(df_results, 'Predicted', 10)
for q in df_q.index:
    if q.left <= pct <= q.right:
        p = df_q[q]

calib_scores = np.abs(df_results['Predicted'].iloc[:-1] - 0.5)
score = abs(pct - 0.5)
pv = np.mean(calib_scores >= score)
asof = datetime.datetime.combine(data.index[-1], time(9,30)) + (periods_30m * timedelta(minutes=30)) 

blob = {
    'Datetime': str(res.index[-1]),
    'IsTrue':df_final['Target_clf'].iloc[-1],
    'Predicted': pct,
    'CalibPredicted': p,
    'Pvalue':pv,
    'ModelNum':periods_30m,
    'AsOf':str(asof)
}

# Write to DB
df_write = pd.DataFrame.from_dict({k:[v] for k, v in blob.items()})


d:\Projects\gamedayspx_lambda\getDailyData.py:243: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return df.groupby(pd.qcut(df[col_name], q))['GreenDay'].mean()
Merging econ data: 100%|██████████| 8/8 [00:00<00:00, 1598.36it/s]
d:\Projects\gamedayspx_lambda\model_intra_v2.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_column] = df[target_column].astype(bool)
d:\Projects\gamedayspx_lambda\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.w

In [4]:
df_write

,Datetime,IsTrue,Predicted,CalibPredicted,Pvalue,ModelNum,AsOf
0,2023-11-22 00:00:00,True,0.712132,0.832636,0.404288,1,2023-11-24 10:00:00


In [ ]:
cursor = connection.cursor()
insert_dataframe_to_sql('results', df_write, cursor)

In [ ]:

if periods_30m > 0:
    regr_blob = {
        'Datetime': str(res.index[-1]),
        'IsTrue':df_final['ClosePct'].iloc[-1],
        'Predicted': regr_pct,
        'Upper': upper,
        'Lower':lower,
        'ModelNum':periods_30m,
        'AsOf':str(asof)
    }
    df_write_reg = pd.DataFrame.from_dict({k:[v] for k, v in regr_blob.items()})
    insert_dataframe_to_sql('reg_results', df_write_reg, cursor)

cursor.close()
connection.close()


In [2]:
regr_blob

{'Datetime': '2023-11-22 00:00:00',
 'IsTrue': 0.0005968736678840791,
 'Predicted': 0.00048111739459897327,
 'Upper': 0.02107334825815718,
 'Lower': -0.018127700802536933,
 'ModelNum': 1,
 'AsOf': '2023-11-24 10:00:00'}

In [ ]:
regr_blob